## # 01. 메일 발송 모듈 제작

###### 앞으로 코드 한 줄의 의미가 이전 수업 보단 더 깊어짐..
###### 지난 시간에 만들었던 메일 발송 모듈 재 사용 실습

In [7]:
# 실습
# 구글 앱비밀번호: uirh ysyy jfia ktlr

# 참조할 라이브러리 정의
import os
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

#--------------------------------------------
# 메일 발송 함수 만들기 (함수(및 이름) 정의)
#--------------------------------------------
def sendMail(from_addr, to_addr, subject, content, files=[]):
    content_type = "plain"

    username = "29alsdud@gmail.com"
    password = "uirh ysyy jfia ktlr"    # (구글 앱 비밀번호)

    smtp = "smtp.gmail.com"
    port = 587

    msg = MIMEMultipart()

    msg['Subject'] = subject
    msg['From'] = from_addr
    msg['To'] = to_addr

    msg.attach(MIMEText(content, content_type))

    if files: # 여기서 file은 리스트임!
        for file_item in files:
            if os.path.exists(file_item):
                with open(file_item, 'rb') as f:
                    basename = os.path.basename(file_item)
                    part = MIMEApplication(f.read(), Name=basename)

                    part['Content-Disposition'] = 'attachment; filename="%s"' % basename
                    msg.attach(part)

                    print(basename, "(이)가 첨부 되었습니다.")


    mail = SMTP(smtp)
    mail.ehlo()
    mail.starttls()
    mail.login(username, password)
    mail.sendmail(from_addr, to_addr, msg.as_string())
    mail.quit()

In [9]:
#--------------------------------------------
# 테스트 코드
#--------------------------------------------
if __name__ == "__main__":

    sendMail("29alsdud@gmail.com", "29alsdud@gmail.com", "오늘 점심 메뉴 추천이욥", "아아, mike check mike check, ok")

## # 02. 메일링 리스트 구현

In [15]:
# .strip(): 줄 마다 있는 줄바꿈 /n 및 공백 제거
# 앞으로 룰: 
    #print(): 코딩 중간 점검용 
        # 일단 해당 줄 까지 코딩 후 출력 해보고
        # 출력 잘 되면 print 앞에 주석 처리 후 쭉 코딩하기

#### 메일 발송 준비

In [2]:
# 모듈 참조
from mylibrary import MyMailer
import datetime as dt

# 날짜 성분값 초기화
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day
print(year, month, day)

2025 10 27


#### 보내는 사람, 메일 제목

In [3]:
# 메일 발신자 정보
fromAddr = "운영지원팀 <29alsdud@gmail.com>"

# 메일 제목을 위한 템플릿
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

#### 메일 본문 가져오기

In [4]:
with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()       # .txt 파일은 utf-8임
    print(contentTmpl)

안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림


#### 수신자 목록 CSV에 대한 반복처리

In [5]:
with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()            # .csv 파일은 utf-8임

    # csv의 라인 수 만큼 반복
    for line in csv:

        # 한 줄 안의 내용 분리
        name, email, file1, file2 = line.strip().split(",")
        #print(name, email, file1, file2)

        toAddr = "{name} <{email}>".format(name = name, email = email)
        #print(toAddr)

        # 메일 제목
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        #print(subject)

        # 메일 내용
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
        #print(content)

        # 메일 보내기
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.


## # 03. 비동기 처리

##### 추후 작업할 crowling 작업을 위한 개념

In [6]:
# 동기 처리는 step-by-step 처리 -> 시간 겁나 소요
# -> 비동기 처리: 다 작업 한번에 start -> 제일 오래 걸리는 작업의 소요 시간이 마지막 작업시간
# -> by .ThreadPollExecutor()
    # ThreadPollExecutor 함수는 .futures() 객체의 내장함수임
    # 동시다발적인 ThreadPollExecutor 작업이 끝나기 전 까지 with문 안에서 대기함.

##### 실습: 메일 발송에 필요한 변수 초기화

In [9]:
# 이전 단원의 소스코드 재사용
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "운영지원팀 <29alsdud@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()

##### 동기식 메일 발송

In [11]:
startTime = dt.datetime.now()

with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()

    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        toAddr = "{name} <{email}>".format(name = name, email = email)
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)

        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
작업에 소요된 시간은 총 19초 입니다.


##### 비동기식 메일 발송

In [ ]:
import concurrent.futures as futures
# 동시 메일 발송 처리

startTime = dt.datetime.now()

with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            toAddr = "{name} <{email}>".format(name = name, email = email)
            subject = subjectTmpl.format(name = name, yy = year, mm = month)
            content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)

            MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptxdocument.pptx (이)가 첨부 되었습니다.
 (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptxdocument.pptx (이)가 첨부 되었습니다.
 (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
작업에 소요된 시간은 총 23초 입니다.


## # 04. 비동기리턴 / 예외처리

### 비동기 리턴

In [ ]:
# 비동기 리턴문 템플릿:
    # 1. '빈 리스트' 만들고
    # 2. '반복문'에서 조건 충족되는거 줍줍하면서
    # 3. 리스트에 조건축종 원소 '쌓는것'

In [ ]:
# What to focus from now on when coding:
    # 1. 사용하는 변수, 객체의 '출처'를 확인하자
    # 2. 결과값의 내용을 확인하자 by 결과 line마다
        # '#print("result=", 변수)'

### 비동기 호출 시 에러처리 방법

In [ ]:
# 05. 딕셔너리의 3.예외처리 참고

# for later: 선택적 crolling
    # chrome browser를 python이 제어한다
    # ex. web-site crolling
    # 중간에 error 났을 때 뻗지 말라고 예외(에러)처리 하는거임

# 데이터 크롤링 작업 때 필수 코드!

#### (∴) 비동기 리턴 w/ 예외처리 = 크롤링 = 빅데이터 수집

##### 실습예제: 비동기 호출로 인한 리턴값 받기

In [41]:
def mywork(n):
    # 비동기로 호출되는 함수가 리턴값을 받을 경우
    processes = []
    # 비동기 프로세스를 저장할 빈 리스트
    with futures.ThreadPoolExecuter(max_workers=3) as executor:
        for n in range(10):
            pro = executor.submit(mywork, n)
            # executor.submit(): 함수를 실행하고, 비동기적 결과를 반환하는 futures 객체를 반환한다.
            processes.append(pro)
            # 비동기 프로세스를 준비한 빈리스트에 저장한다
        
        for p in processes:
            result = p.result()
            # 비동기 처리된 리턴값을 받는다.

##### 실습1-1: 비동기 작업을 위함 함수 정의

In [42]:
import random
# 내장함수임
# random module: provides functions for generating pseudo-random numbers and performing random selections.

def randomwork(name):
    randomSecond = random.randrange(1, 9)
    # random.randrange(start, stop, step): This function returns a randomly selected element from range(start, stop, step). The step argument is optional.

    print("[%s] 작업을 %d초 동안 수행합니다." %(name, randomSecond))

    for i in range(0, randomSecond):
        time.sleep(1)
        print("[%s] 작업을 %d초 동안 수행합니다." %(name, i + 1))

    print("[%s] 작업이 종료 되었습니다.." % name)

    return randomSecond
    # 실제로 몇초동안 작업 했는지 리턴

##### 실습1-2: 비동기 작업 처리 결과 후 결과값 리턴 받기

In [ ]:
startTime = dt.datetime.now()

names = ['Lee', "Kim", 'Hong', "Park", 'Nam']
processes = []
#비동기 프로세스를 저장할 껍데기
resultSet = []
# 비동기 작업 결과를 저장할 껍데기

with futures.ThreadPoolExecutor(max_workers=3) as executor:
    for n in names:
        pro = executor.submit(randomwork, n)
        # 비동기 작업 생성
        processes.append(pro)
    #print("result=", processes)

    for p in processes:
        result = p.result()
        # 비동기 작업한 randomWork 함수의 결과값 리턴받음
        resultSet.append(result)
    #print("result=", reultSet)

print("비동기 처리가 총 %d초의 작업을 수행 하였습니다." % sum(resultSet))
# 동기처리 했을 때를 가정한 실행시간 출력

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)
# 실제 실행시간 (비동기 처리) 출력

[Lee] 작업을 3초 동안 수행합니다.
[Kim] 작업을 1초 동안 수행합니다.
[Hong] 작업을 3초 동안 수행합니다.
[Lee] 작업을 1초 동안 수행합니다.
[Hong] 작업을 1초 동안 수행합니다.
[Kim] 작업을 1초 동안 수행합니다.
[Kim] 작업이 종료 되었습니다..
[Park] 작업을 3초 동안 수행합니다.
[Lee] 작업을 2초 동안 수행합니다.
[Hong] 작업을 2초 동안 수행합니다.
[Park] 작업을 1초 동안 수행합니다.
[Lee] 작업을 3초 동안 수행합니다.[Hong] 작업을 3초 동안 수행합니다.
[Hong] 작업이 종료 되었습니다..
[Nam] 작업을 5초 동안 수행합니다.
[Park] 작업을 2초 동안 수행합니다.

[Lee] 작업이 종료 되었습니다..
[Nam] 작업을 1초 동안 수행합니다.[Park] 작업을 3초 동안 수행합니다.
[Park] 작업이 종료 되었습니다..

[Nam] 작업을 2초 동안 수행합니다.
[Nam] 작업을 3초 동안 수행합니다.
[Nam] 작업을 4초 동안 수행합니다.
[Nam] 작업을 5초 동안 수행합니다.
[Nam] 작업이 종료 되었습니다..
비동기 처리가 총 15초의 작업을 수행 하였습니다.
작업에 소요된 시간은 총 8초 입니다.


##### 실습 2-1: 비동기 '예외'처리

In [ ]:
# 예외(=에러)를 발생시키는 함수 정의

import random
seasons = ['봄', '여름', '가을', '겨울']

def seasonWork(name):
    randomSecond = random.randrange(1, 9)
    # 내장함수
    # random.randrange(start, stop, step): This function returns a randomly selected element from range(start, stop, step). The step argument is optional.

    print("[%s] 작업을 %d초 동안 수행합니다." %(name, randomSecond))

    for i in range(0, randomSecond):
        time.sleep(1)
        print("[%s] 지금은 <%s>입니다." %(name, seasons[i]))
        # 에러사항을 위한 모델임 (i가 4 이상부터 에러발생)

    print("[%s] 작업이 종료 되었습니다." % name)

    return [name, randomSecond]

##### 실습2-2: 에러를 포함한 작업 처리 결과 후 결과값 리턴 받기

In [44]:
startTime = dt.datetime.now()

names = ['Lee', "Kim", 'Hong', "Park", 'Nam']
processes = []
nameSet = []
timeSet = []

with futures.ThreadPoolExecutor(max_workers=len(names)) as executor:
    for n in names:
        pro = executor.submit(randomwork, n)
        processes.append(pro)
    #print("result=", processes)

#-----------------------------------------------------
# 에러 대비용 try ~ except 구문!
#-----------------------------------------------------
    for p in processes:
        try:
            name, n = p.result()
            nameSet.append(name)
            timeSet.append(n)
            print("%s의 작업이 완료 되었습니다." % name)
        except Exception as e:
            print(e)
#-----------------------------------------------------
endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[Lee] 작업을 8초 동안 수행합니다.
[Kim] 작업을 1초 동안 수행합니다.
[Hong] 작업을 6초 동안 수행합니다.
[Park] 작업을 8초 동안 수행합니다.
[Nam] 작업을 3초 동안 수행합니다.
[Hong] 작업을 1초 동안 수행합니다.
[Park] 작업을 1초 동안 수행합니다.
[Nam] 작업을 1초 동안 수행합니다.
[Lee] 작업을 1초 동안 수행합니다.
[Kim] 작업을 1초 동안 수행합니다.
[Kim] 작업이 종료 되었습니다..
[Hong] 작업을 2초 동안 수행합니다.[Park] 작업을 2초 동안 수행합니다.

[Nam] 작업을 2초 동안 수행합니다.
[Lee] 작업을 2초 동안 수행합니다.
[Park] 작업을 3초 동안 수행합니다.
[Lee] 작업을 3초 동안 수행합니다.
[Nam] 작업을 3초 동안 수행합니다.
[Nam] 작업이 종료 되었습니다..
[Hong] 작업을 3초 동안 수행합니다.
[Park] 작업을 4초 동안 수행합니다.
[Lee] 작업을 4초 동안 수행합니다.
[Hong] 작업을 4초 동안 수행합니다.
[Park] 작업을 5초 동안 수행합니다.
[Hong] 작업을 5초 동안 수행합니다.
[Lee] 작업을 5초 동안 수행합니다.
[Park] 작업을 6초 동안 수행합니다.
[Lee] 작업을 6초 동안 수행합니다.
[Hong] 작업을 6초 동안 수행합니다.
[Hong] 작업이 종료 되었습니다..
[Park] 작업을 7초 동안 수행합니다.
[Lee] 작업을 7초 동안 수행합니다.
[Park] 작업을 8초 동안 수행합니다.
[Park] 작업이 종료 되었습니다..
[Lee] 작업을 8초 동안 수행합니다.
[Lee] 작업이 종료 되었습니다..
cannot unpack non-iterable int object
cannot unpack non-iterable int object
cannot unpack non-iterable int object
cannot unpack non-iterable int object
cannot unpac